In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pyspark
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, when,mean,count, col,desc,year, quarter, when,lit, to_date, to_timestamp, concat, avg,countDistinct
from pyspark.sql.types import StringType
from pyspark.sql.functions import regexp_replace
from pyspark.ml.feature import StringIndexer
import pandas as pd
import numpy as np

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 28.4 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824025 sha256=225b731bc55d673840276a1b53fa1a7bc528527789d19443fda562883e59ea27
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark


In [ ]:
# Creating Spark and setting a name for the application
spark=SparkSession.builder.appName('Project').config("spark.driver.memory", "4g").getOrCreate()
conf = spark.sparkContext.getConf()
for key, value in conf.getAll():
    print(f"{key} = {value}")

spark.driver.extraJavaOptions = -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED
spark.driver.port = 44261
spark.driver.memory = 4g
spark.executor.id = driver
spark.app.submitTime = 1678797657386
spark.app.startTime = 1678797657534
spark.driver.host = d2028b7965c8
spark.app.id = local-1678797658779
spark.rdd.compress = Tr

In [ ]:
music = spark.read.csv("/content/drive/MyDrive/Project_Data/final_spotify.csv",inferSchema=True, header=True)

In [ ]:
# Displaying the dataset
music.show()

+--------+--------------------+--------------------+--------------------+--------------------+----------+-----------+--------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+-----------+--------------------+
|index_id|            track_id|             artists|          album_name|          track_name|popularity|duration_ms|explicit|danceability|energy|key|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence|  tempo|time_signature|track_genre|spotify_release_date|
+--------+--------------------+--------------------+--------------------+--------------------+----------+-----------+--------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+-----------+--------------------+
|       0|5SuOikwiRyPMVoIQD...|         Gen Hoshino|              Comedy|              Comedy|        73|     230666|   FALSE|       0.676| 0.461|  1|  -6.746| 

by overlooking at the above data we can drop columns such as ["track_id","track_name","explicit","spotify_release_date"] because they have no business with our target variable

In [ ]:
music = music.drop("track_id","track_name","explicit","spotify_release_date")
music.show()

+--------+--------------------+--------------------+----------+-----------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+-----------+
|index_id|             artists|          album_name|popularity|duration_ms|danceability|energy|key|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence|  tempo|time_signature|track_genre|
+--------+--------------------+--------------------+----------+-----------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+-----------+
|       0|         Gen Hoshino|              Comedy|        73|     230666|       0.676| 0.461|  1|  -6.746|   0|      0.143|      0.0322|         1.01E-6|   0.358|  0.715| 87.917|           4.0|   acoustic|
|       1|        Ben Woodward|    Ghost (Acoustic)|        55|     149610|        0.42| 0.166|  1| -17.235|   1|     0.0763|       0.924|         5.56E-6|   0.101|  0.

in the dataset there is a column name "instrumentalness" whose values needs to be transformed for further operations

In [ ]:
from pyspark.sql.functions import col, format_number
music = music.drop("instrumentalness_decimal")
music = music.withColumn("instrumentalness_decimal", format_number(col("instrumentalness"), 20))
music = music.drop("instrumentalness")
music.show(1,truncate = False)


+--------+-----------+----------+----------+-----------+------------+------+---+--------+----+-----------+------------+--------+-------+------+--------------+-----------+------------------------+
|index_id|artists    |album_name|popularity|duration_ms|danceability|energy|key|loudness|mode|speechiness|acousticness|liveness|valence|tempo |time_signature|track_genre|instrumentalness_decimal|
+--------+-----------+----------+----------+-----------+------------+------+---+--------+----+-----------+------------+--------+-------+------+--------------+-----------+------------------------+
|0       |Gen Hoshino|Comedy    |73        |230666     |0.676       |0.461 |1  |-6.746  |0   |0.143      |0.0322      |0.358   |0.715  |87.917|4.0           |acoustic   |0.00000101000000000000  |
+--------+-----------+----------+----------+-----------+------------+------+---+--------+----+-----------+------------+--------+-------+------+--------------+-----------+------------------------+
only showing top 1 r

In [ ]:
music.select("instrumentalness_decimal").show(truncate=False)

+------------------------+
|instrumentalness_decimal|
+------------------------+
|0.00000101000000000000  |
|0.00000556000000000000  |
|0.00000000000000000000  |
|0.00007070000000000000  |
|0.00000000000000000000  |
|0.00000000000000000000  |
|0.00000289000000000000  |
|0.00000000000000000000  |
|0.00000000000000000000  |
|0.00419000000000000000  |
|0.00000000000000000000  |
|0.00000000000000000000  |
|0.00000000000000000000  |
|0.00000000000000000000  |
|0.00002830000000000000  |
|0.00004560000000000000  |
|0.00000403000000000000  |
|0.00001170000000000000  |
|0.00000000000000000000  |
|0.00000000000000000000  |
+------------------------+
only showing top 20 rows



In [ ]:
music = music.drop("index_id")

In [ ]:
music.groupBy("track_genre").count().show(20)

print("by looking at the output \n in our TARGET VARIABLE there are so many misleading or false values are there \n they could lead towards to the inaccurate predictions \n so we need to clear those rows containing false data.")

+--------------+-----+
|   track_genre|count|
+--------------+-----+
|         anime| 1000|
|          folk| 1000|
|             3|   23|
|     hardstyle| 1000|
|         0.576|    1|
|   alternative| 1000|
|   death-metal| 1000|
|        74.077|    1|
|detroit-techno| 1000|
|           idm| 1000|
|       105.188|    1|
|         k-pop| 1000|
|       j-dance| 1000|
|        68.958|    2|
|       ambient| 1000|
|        guitar|  999|
|          goth| 1000|
|      cantopop| 1000|
|         blues| 1000|
|             5|   12|
+--------------+-----+
only showing top 20 rows

by looking at the output 
 in our TARGET VARIABLE there are so many misleading or false values are there 
 they could lead towards to the inaccurate predictions 
 so we need to clear those rows containing false data.


In [ ]:
drop_list = ["3", "0.576", "74.077","105.188","68.958","5","68.453","151.539","117.11","76.791","89.01","1","125.262","4","148.759","114.211","7","0.159","0.983","0.103","89.912","0.0898","81.078","60.015","1.50E-05","76.691","95.073","121.165","0.34","134.113","0.114","90.051","91.467","131.721","133.67","10","133.106","-12.753"]
music_df = music.filter(~music.track_genre.isin(drop_list))
music_df.groupBy("track_genre").count().show()

+--------------+-----+
|   track_genre|count|
+--------------+-----+
|         anime| 1000|
|          folk| 1000|
|     hardstyle| 1000|
|   alternative| 1000|
|   death-metal| 1000|
|detroit-techno| 1000|
|           idm| 1000|
|         k-pop| 1000|
|       j-dance| 1000|
|       ambient| 1000|
|        guitar|  999|
|          goth| 1000|
|      cantopop| 1000|
|         blues| 1000|
|     breakbeat| 1000|
|         dance| 1000|
|        groove| 1000|
|        indian| 1000|
|        german|  991|
|        french| 1000|
+--------------+-----+
only showing top 20 rows



In [ ]:
print(music_df.show(1))

+-----------+----------+----------+-----------+------------+------+---+--------+----+-----------+------------+--------+-------+------+--------------+-----------+------------------------+
|    artists|album_name|popularity|duration_ms|danceability|energy|key|loudness|mode|speechiness|acousticness|liveness|valence| tempo|time_signature|track_genre|instrumentalness_decimal|
+-----------+----------+----------+-----------+------------+------+---+--------+----+-----------+------------+--------+-------+------+--------------+-----------+------------------------+
|Gen Hoshino|    Comedy|        73|     230666|       0.676| 0.461|  1|  -6.746|   0|      0.143|      0.0322|   0.358|  0.715|87.917|           4.0|   acoustic|    0.000001010000000...|
+-----------+----------+----------+-----------+------------+------+---+--------+----+-----------+------------+--------+-------+------+--------------+-----------+------------------------+
only showing top 1 row

None


now we need to convert our categorical columns into numeric for further operations
we will do it using **StringIndexer**

In [ ]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler

In [ ]:
indexer = StringIndexer(inputCol="artists",outputCol="artists_Num")
#if any error occurs just uncomment the below line
#music_df=music_df.drop("artists_Num")
music_df = indexer.fit(music_df).transform(music_df)
music_df = music_df.withColumn("artists_Num", col("artists_Num").cast("integer"))

In [ ]:
indexer = StringIndexer(inputCol="album_name",outputCol="album_name_Num")
#if any error occurs just uncomment the below line
#music_df=music_df.drop("album_name_Num")
music_df = indexer.fit(music_df).transform(music_df)
music_df = music_df.withColumn("album_name", col("album_name").cast("integer"))

now that we have numeric columns ,drop original columns

In [ ]:
music_df_numeric = music_df.drop("artists","album_name")
music_df_numeric.show()

+----------+-----------+------------+------+---+--------+----+-----------+------------+--------+-------+-------+--------------+-----------+------------------------+-----------+--------------+
|popularity|duration_ms|danceability|energy|key|loudness|mode|speechiness|acousticness|liveness|valence|  tempo|time_signature|track_genre|instrumentalness_decimal|artists_Num|album_name_Num|
+----------+-----------+------------+------+---+--------+----+-----------+------------+--------+-------+-------+--------------+-----------+------------------------+-----------+--------------+
|        73|     230666|       0.676| 0.461|  1|  -6.746|   0|      0.143|      0.0322|   0.358|  0.715| 87.917|           4.0|   acoustic|    0.000001010000000...|       2100|        4670.0|
|        55|     149610|        0.42| 0.166|  1| -17.235|   1|     0.0763|       0.924|   0.101|  0.267| 77.489|           4.0|   acoustic|    0.000005560000000...|       1492|       12573.0|
|        57|     210826|       0.438| 0.

# now only thing remaining is to convert our target variable into numeric

In [ ]:
indexer = StringIndexer(inputCol="track_genre",outputCol="track_genre_Num")
#if any error occurs just uncomment the below line
#music_df_numeric=music_df_numeric.drop("track_genre_Num")
music_df_numeric = indexer.fit(music_df_numeric).transform(music_df_numeric)
music_df_numeric = music_df_numeric.withColumn("track_genre_Num", col("track_genre_Num").cast("integer"))

after converting drop original column

In [ ]:
music_df_numeric = music_df_numeric.drop("track_genre")
music_df_numeric.show()

+----------+-----------+------------+------+---+--------+----+-----------+------------+--------+-------+-------+--------------+------------------------+-----------+--------------+---------------+
|popularity|duration_ms|danceability|energy|key|loudness|mode|speechiness|acousticness|liveness|valence|  tempo|time_signature|instrumentalness_decimal|artists_Num|album_name_Num|track_genre_Num|
+----------+-----------+------------+------+---+--------+----+-----------+------------+--------+-------+-------+--------------+------------------------+-----------+--------------+---------------+
|        73|     230666|       0.676| 0.461|  1|  -6.746|   0|      0.143|      0.0322|   0.358|  0.715| 87.917|           4.0|    0.000001010000000...|       2100|        4670.0|             98|
|        55|     149610|        0.42| 0.166|  1| -17.235|   1|     0.0763|       0.924|   0.101|  0.267| 77.489|           4.0|    0.000005560000000...|       1492|       12573.0|             98|
|        57|     210

In [ ]:
music_df_numeric.printSchema()

root
 |-- popularity: string (nullable = true)
 |-- duration_ms: string (nullable = true)
 |-- danceability: string (nullable = true)
 |-- energy: string (nullable = true)
 |-- key: string (nullable = true)
 |-- loudness: string (nullable = true)
 |-- mode: string (nullable = true)
 |-- speechiness: string (nullable = true)
 |-- acousticness: string (nullable = true)
 |-- liveness: string (nullable = true)
 |-- valence: string (nullable = true)
 |-- tempo: double (nullable = true)
 |-- time_signature: double (nullable = true)
 |-- instrumentalness_decimal: string (nullable = true)
 |-- artists_Num: integer (nullable = true)
 |-- album_name_Num: double (nullable = false)
 |-- track_genre_Num: integer (nullable = true)



observing above schema there are still so many columns with String type we need to convert them first for the further operations

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType
# define a UDF to convert multiple string columns to double columns
string_to_double = udf(lambda x: float(x) if x else None, DoubleType())
string_columns = ["popularity", "duration_ms", "danceability", "energy","album_name_Num","track_genre_Num", "key", "loudness","artists_Num","mode", "speechiness", "acousticness", "liveness", "valence","instrumentalness_decimal"]

In [ ]:
for column in string_columns:
    music_df_numeric = music_df_numeric.withColumn(column + "", string_to_double(column))
music_df_numeric.printSchema()

root
 |-- popularity: double (nullable = true)
 |-- duration_ms: double (nullable = true)
 |-- danceability: double (nullable = true)
 |-- energy: double (nullable = true)
 |-- key: double (nullable = true)
 |-- loudness: double (nullable = true)
 |-- mode: double (nullable = true)
 |-- speechiness: double (nullable = true)
 |-- acousticness: double (nullable = true)
 |-- liveness: double (nullable = true)
 |-- valence: double (nullable = true)
 |-- tempo: double (nullable = true)
 |-- time_signature: double (nullable = true)
 |-- instrumentalness_decimal: double (nullable = true)
 |-- artists_Num: double (nullable = true)
 |-- album_name_Num: double (nullable = true)
 |-- track_genre_Num: double (nullable = true)



In [ ]:
from pyspark.sql.functions import isnan, when, count, col
#music_df_numeric.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in music_df_numeric.columns]).show()
# drop rows with missing values
music_df_numeric = music_df_numeric.dropna()

# lets check correlation of every feature with our target variable

In [ ]:
# Compute the correlation coefficients with target columns
from pyspark.sql.functions import corr
def print_correlation_with_target(music_df_numeric, target):
    columns = ["popularity", "duration_ms", "danceability", "energy", "key", "loudness","mode", "speechiness", "acousticness", "liveness", "valence","instrumentalness_decimal","artists_Num","album_name_Num","track_genre_Num"]
    for column in columns:
      correlation = music_df_numeric.select(corr(column, target)).collect()[0][0]
      print(f"{column}\t correlation with target\t : {correlation}")
target="track_genre_Num"
print_correlation_with_target(music_df_numeric,target)

Py4JJavaError: ignored

# Now, lets build RANDOM FOREST Classifier model

In [ ]:
from pyspark.ml.feature import VectorAssembler
# Define the input columns to the model
input_cols = string_columns = ["popularity", "duration_ms", "danceability", "energy","album_name_Num", "key", "loudness","artists_Num","mode", "speechiness", "acousticness", "liveness", "valence","instrumentalness_decimal"]
# Create a VectorAssembler object to transform input features into a single vector
assembler = VectorAssembler(inputCols=input_cols, outputCol='features')

In [ ]:
# Transform the input data using VectorAssembler
data = assembler.transform(music_df_numeric).select('features', 'track_genre_Num')

In [ ]:
# Split the data into training and testing sets
train_data, test_data = data.randomSplit([0.7, 0.3], seed=1234)

In [ ]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Define the random forest classifier model
rf = RandomForestClassifier(featuresCol="features", labelCol="track_genre_Num", numTrees=100)
# Train the model on the training data
rf_model = rf.fit(train_data)
# Make predictions on the test data
predictions_by_RF = rf_model.transform(test_data)
#Evaluate the model using the accuracy metric
evaluator = BinaryClassificationEvaluator(labelCol="track_genre_Num", metricName="areaUnderROC")
accuracy = evaluator.evaluate(predictions_by_RF)
print("Accuracy = {:.2f}".format(accuracy))
